In [ ]:
!pip install flask_ngrok
!pip install flask

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import GridSearchCV

import pickle

dataset = datasets.load_iris()

**Let's know our dataset**

![alt text](https://www.w3resource.com/w3r_images/iris_flower_dataset.png)

**How Dataset was collected?**

![alt text](https://www.oreilly.com/library/view/python-artificial-intelligence/9781789539462/assets/462dc4fa-fd62-4539-8599-ac80a441382c.png)

In [ ]:
dataset.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [ ]:
x = dataset.data
y = dataset.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, 
                                                    random_state=1)

In [ ]:
model = SVC()
options = {
    "kernel": ("linear", "poly", "rbf"),
    "degree": [3, 7, 10, 15, 50],
    "gamma": ("scale", "auto"),
    "C": [0.1, 1, 10, 100]
}
gs = GridSearchCV(model, options)
gs.fit(x_train, y_train)
y_pred = gs.predict(x_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred) 
print("Mean Square Error : ", mse)

acc_score = accuracy_score(y_test, y_pred)
print("Accuracy: " + str(acc_score))

Mean Square Error :  0.022222222222222223
Accuracy: 0.9777777777777777


In [ ]:
pickle.dump(gs.best_estimator_, open('/content/model.pkl', 'wb'))

# New File

In [ ]:
from flask import Flask, jsonify, make_response, request
from flask_ngrok import run_with_ngrok
import pickle
import numpy as np

model = pickle.load(open("/content/model.pkl", 'rb'))

app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
model.predict([[5.1, 3.5, 1.4, 0.2]])[0]

0

In [ ]:
@app.route('/', methods=['GET'])
def index():
  return ("Server is up and running...")

'''
 {
    "sepalLength": 5,
    "sepalWidth":  6,
    "petalLength": 6.8,
    "petalWidth":  1
  }
'''

@app.route('/predict', methods=['POST'])
def predict():
  data = request.get_json(force = True)
  param1 = data["sepalLength"]
  param2 = data['sepalWidth']
  param3 = data['petalLength']
  param4 = data['petalWidth']
  predict_on = np.array([[param1, param2, param3, param4]])
  y_predict = model.predict(predict_on)
  print(y_predict[0])
  return jsonify(results = int(y_predict[0]))
'''
  {
    "results" : [0]
  }
'''
app.run()

In [ ]:
# For Deployment

import sklearn
sklearn.__version__

'0.22.2.post1'